# Model evaluation

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [37]:
from fastai.vision import *
from fastai.metrics import error_rate
from fastai.widgets import *
import torch
torch.cuda.get_device_properties(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")

## How well can we predict genera training on species?

In [3]:
bs = 32
path = Path("/home/alan/Desktop/personal_professional/fastai/mushroom_classifier")
#ds_tfms = get_transforms(flip_vert=True, max_rotate=45.)

In [19]:
data = (ImageList.from_csv(path / "images/320",
                          "../../resources/ds4_min5img.csv", cols="image_id",
                          suffix='.jpg')
        .split_from_df("is_validation")
        .label_from_df("text_name")
#        .transform(tfms=ds_tfms, size=300)
        .transform(tfms=None,size=300)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [20]:
learn = cnn_learner(data, models.resnet34, metrics=[accuracy, top_k_accuracy])

In [21]:
learn.load("../../../resources/models/resnet34_ds4_min5img")

Learner(data=ImageDataBunch;

Train: LabelList (517331 items)
x: ImageList
Image (3, 300, 300),Image (3, 300, 300),Image (3, 300, 300),Image (3, 300, 300),Image (3, 300, 300)
y: CategoryList
Physarum sulphureum,Tricholoma murrillianum,Kuehneromyces marginellus,unknown,Flammulaster muricatus
Path: /home/alan/Desktop/personal_professional/fastai/mushroom_classifier/images/320;

Valid: LabelList (129338 items)
x: ImageList
Image (3, 300, 300),Image (3, 300, 300),Image (3, 300, 300),Image (3, 300, 300),Image (3, 300, 300)
y: CategoryList
Psilocybe tasmaniana,Inocybe dulciolens,Pilophorus clavatus,Marasmius pulcherripes,Phaeolus schweinitzii
Path: /home/alan/Desktop/personal_professional/fastai/mushroom_classifier/images/320;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool

In [22]:
len(data.valid_ds.x)

129338

In [43]:
data.classes

['Abortiporus biennis',
 'Abrothallus welwitschii',
 'Acanthobasidium penicillatum',
 'Acanthophysellum canadense',
 'Acanthophysellum lividocoeruleum',
 'Acarospora americana',
 'Acarospora elevata',
 'Acarospora fuscata',
 'Acarospora nodulosa',
 'Acarospora obnubila',
 'Acarospora obpallens',
 'Acarospora placodiiformis',
 'Acarospora rosulata',
 'Acarospora schleicheri',
 'Acarospora socialis',
 'Acarospora strigata',
 'Acrocordia conoidea',
 'Acrocordia gemmata',
 'Aculops rhois',
 'Aegerita candida',
 'Agaricus "tawny-gold"',
 'Agaricus abruptibulbus',
 'Agaricus agrinferus',
 'Agaricus albolutescens',
 'Agaricus alligator',
 'Agaricus amicosus',
 'Agaricus andrewii',
 'Agaricus argenteus',
 'Agaricus arorae',
 'Agaricus arvensis',
 'Agaricus augustus',
 'Agaricus auricolor',
 'Agaricus austrovinaceus',
 'Agaricus benesii',
 'Agaricus bernardi',
 'Agaricus bisporus',
 'Agaricus bitorquis',
 'Agaricus braendlei',
 'Agaricus bresadolanus',
 'Agaricus brunneofibrillosus',
 'Agaricus

In [53]:
with torch.no_grad():
    for data_ in data.valid_dl:
        images, labels = data_
        learn.pred_batch(images)
        break

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/alan/anaconda3/envs/mushroom_api/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/alan/anaconda3/envs/mushroom_api/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/alan/anaconda3/envs/mushroom_api/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/alan/anaconda3/envs/mushroom_api/lib/python3.6/site-packages/fastai/data_block.py", line 654, in __getitem__
    x = x.apply_tfms(self.tfms, **self.tfmargs)
AttributeError: 'Tensor' object has no attribute 'apply_tfms'


In [47]:
class_correct = list(0. for i in range(data.c))
class_total = list(0. for i in range(data.c))
with torch.no_grad():
    for j, data_ in enumerate(data.valid_dl):
        if j > 10:
            break
        print(j)
        images, labels = data_
        outputs = learn.pred_batch(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted.cpu() == labels.cpu()).squeeze()
        for i in range(bs):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(data.c):
    print('Accuracy of %s : %2d %%' % (
        data.classes[i], 100 * class_correct[i] / class_total[i]) if class_total[i] > 0 else 0)

0


AttributeError: 'Tensor' object has no attribute 'apply_tfms'

In [ ]:
from sklearn.metrics import precision_score
def precision(trues, preds):
    return precision_score(trues, preds)